<a href="https://colab.research.google.com/github/prajagopal1992/Projects-2023/blob/main/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
!pip install datasets transformers huggingface_hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 70.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 55.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 104.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 21.9 MB/s eta 0:00:00


In [ ]:
!apt -get install git-lfs

E: Command line option 'g' [from -get] is not understood in combination with the other options.


In [ ]:
#Preprocessing data

from datasets import load_dataset
imdb = load_dataset("imdb")

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
small_train_dataset = imdb['train'].shuffle(seed=42).select([i for i in list(range(3000))])
small_test_dataset = imdb['test'].shuffle(seed=42).select([i for i in list(range(300))])

In [ ]:
#Using DistilBERT Tokenizer

from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
#Preparing text inputs for the model for both train and test datasets

def preprocess_function(examples):
  return tokenizer(examples['text'], truncation=True)

tokenized_train = small_train_dataset.map(preprocess_function, batched=True)
tokenized_test = small_test_dataset.map(preprocess_function, batched=True)

#Use data_collator to convert training sample to PyTorch tensors and 
#concatenate with padding

from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

In [ ]:
#Replace pretraining head with classification head
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'classifi

In [ ]:
#defining metrics to evaluate model (accuracy and f1 score)
import numpy as np
from datasets import load_metric

def compute_metrics(eval_pred):
  load_accuracy = load_metric("accuracy")
  load_f1 = load_metric('f1')

  logits, labels = eval_pred
  predictions = np.argmax(logits, axis=-1)
  accuracy = load_accuracy.compute(predictions=predictions,
                                   references=labels)['accuracy']
  f1 = load_f1.compute(predictions=predictions,
                       references=labels)["f1"]
  return {"accuracy":accuracy,"f1":f1}

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
#Define training arguments and define trainer with all the constructed objects
from transformers import TrainingArguments, Trainer

repo_name = "finetuning-sentiment-model-3000-samples"

training_args = TrainingArguments(
    output_dir=repo_name,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    save_strategy="epoch",
    push_to_hub=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

Cloning https://huggingface.co/prajagopal1992/finetuning-sentiment-model-3000-samples into local empty directory.


Download file pytorch_model.bin:   0%|          | 15.4k/255M [00:00<?, ?B/s]

Download file training_args.bin: 100%|##########| 3.56k/3.56k [00:00<?, ?B/s]

Download file runs/Apr22_20-18-24_efb7f1bbd1d7/events.out.tfevents.1682194725.efb7f1bbd1d7.157.0: 100%|#######…

Download file runs/Apr22_20-18-24_efb7f1bbd1d7/1682194725.794019/events.out.tfevents.1682194725.efb7f1bbd1d7.1…

...

Download file runs/Apr22_20-18-24_efb7f1bbd1d7/events.out.tfevents.1682195058.efb7f1bbd1d7.157.2: 100%|#######…

Clean file runs/Apr22_20-18-24_efb7f1bbd1d7/events.out.tfevents.1682194725.efb7f1bbd1d7.157.0:  24%|##3       …

Clean file runs/Apr22_20-18-24_efb7f1bbd1d7/1682194725.794019/events.out.tfevents.1682194725.efb7f1bbd1d7.157.…

Clean file runs/Apr22_20-18-24_efb7f1bbd1d7/events.out.tfevents.1682195058.efb7f1bbd1d7.157.2: 100%|##########…

Clean file pytorch_model.bin:   0%|          | 1.00k/255M [00:00<?, ?B/s]

In [ ]:
#funetune the model on the dataset
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss


TrainOutput(global_step=376, training_loss=0.30369243216007313, metrics={'train_runtime': 307.0266, 'train_samples_per_second': 19.542, 'train_steps_per_second': 1.225, 'total_flos': 783875831546880.0, 'train_loss': 0.30369243216007313, 'epoch': 2.0})

In [ ]:
#Now that the DistilBERT model is trained for sentiment analysis,
#time to evaluate the model
trainer.evaluate()

<ipython-input-9-aba06c9f2ac0>:6: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  load_accuracy = load_metric("accuracy")


{'eval_loss': 0.2804046869277954,
 'eval_accuracy': 0.8733333333333333,
 'eval_f1': 0.8741721854304636,
 'eval_runtime': 5.4115,
 'eval_samples_per_second': 55.438,
 'eval_steps_per_second': 3.511,
 'epoch': 2.0}

In [ ]:
#Analyzing new data with the model -- begin with uploading the model

trainer.push_to_hub()

To https://huggingface.co/prajagopal1992/finetuning-sentiment-model-3000-samples
   1063473..9581f23  main -> main

   1063473..9581f23  main -> main

To https://huggingface.co/prajagopal1992/finetuning-sentiment-model-3000-samples
   9581f23..ecaf529  main -> main

   9581f23..ecaf529  main -> main



'https://huggingface.co/prajagopal1992/finetuning-sentiment-model-3000-samples/commit/9581f23fcc67f287fbbf3ad2599450b80433e683'

In [ ]:
#Classify 2 new movie reviews using the model

from transformers import pipeline

sentiment_model = pipeline(
    model= "prajagopal1992/finetuning-sentiment-model-3000-samples")
sentiment_model(["I love this move", "This movie sucks!"])

[{'label': 'LABEL_1', 'score': 0.9431331157684326},
 {'label': 'LABEL_0', 'score': 0.9703739285469055}]

In [ ]:
#In the IMDB dataset, Label 1 means positive and Label 0. So the model seems
#to be working well